# Modello LSMT (Long Short-Term Memory) 

Primo step split del dataset

In [4]:
import pandas as pd
import glob
import os
import gc
from collections import Counter
from sklearn.preprocessing import StandardScaler
import joblib

In [ ]:
INPUT_DIR = '../Pre-Elaborazione Dati/Dataset'
all_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

TRAIN_FILES = all_files[0:16]
VAL_FILES = all_files[16:20]
TEST_FILES = all_files[20:24]
print(f"File di Training: {len(TRAIN_FILES)}")
print(f"File di Validazione: {len(VAL_FILES)}")
print(f"File di Test: {len(TEST_FILES)}")

print(f"\nPrimo file di Train: {os.path.basename(TRAIN_FILES[0])}")
print(f"Primo file di Val: {os.path.basename(VAL_FILES[0])}")
print(f"Primo file di Test: {os.path.basename(TEST_FILES[0])}")

File di Training: 16
File di Validazione: 4
File di Test: 4

Primo file di Train: blocco_000-segmentato.parquet
Primo file di Val: blocco_016-segmentato.parquet
Primo file di Test: blocco_020-segmentato.parquet


In [ ]:
INPUT_DIR = '../Pre-Elaborazione Dati/Dataset' 

all_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

if len(all_files) == 0:
    print(f"ERRORE CRITICO: Nessun file .parquet trovato in '{INPUT_DIR}'")
else:
    print(f"Trovati {len(all_files)} file .parquet in '{INPUT_DIR}'")

TRAIN_FILES = all_files[0:16]
IPERPARAMETRI_TIMESTEP = [10, 15, 20, 25, 30, 40]

print(f"\nFASE 1: Avvio scansione unificata (UN FILE ALLA VOLTA)...")

total_counts_counter = Counter()
total_ids_set = set()

for file_path in TRAIN_FILES:
    print(f"  Processando {os.path.basename(file_path)}...")
    try:
        df = pd.read_parquet(file_path, columns=['TrajectoryID'])
        
        total_ids_set.update(df['TrajectoryID'].unique())
        
        total_counts_counter.update(df['TrajectoryID'])
        
        del df
        gc.collect()
    except Exception as e:
        print(f"    ERRORE nel leggere {file_path}: {e}")
        

print("FASE 2: Verifica e Analisi.")

conteggio_set = len(total_ids_set)
conteggio_counter = len(total_counts_counter)

print(f"\VERIFICA CONTEGGI:")
print(f"ID Unici (metodo 'set'):     {conteggio_set:,}")
print(f"ID Unici (metodo 'Counter'): {conteggio_counter:,}")

if conteggio_set != conteggio_counter:
    print("ERRORE LOGICO: I conteggi non corrispondono. C'è ancora un problema.")
else:
    print("\nSUCCESSO! I conteggi corrispondono.")
    
    all_lengths = list(total_counts_counter.values())
    totale_traiettorie_analisi = conteggio_counter

    print(f"\nRisultati dell'analisi (su {totale_traiettorie_analisi:,} traiettorie totali di TRAINING):")
    print("-" * 75)
    print(f"{'Window Size':<15} | {'Traiettorie Usabili':<20} | {'Traiettorie Scartate':<20} | {'% Usabili':<10}")
    print("-" * 75)

    for timestep in IPERPARAMETRI_TIMESTEP:
        traiettorie_usabili = sum(1 for length in all_lengths if length >= timestep)
        traiettorie_scartate = totale_traiettorie_analisi - traiettorie_usabili
        percentuale_usabili = (traiettorie_usabili / totale_traiettorie_analisi) * 100
        print(f"{timestep:<15} | {traiettorie_usabili:<20} | {traiettorie_scartate:<20} | {percentuale_usabili:<10.2f}%")

    print("-" * 75)
    print("Analisi completata.")

<>:46: SyntaxWarning: invalid escape sequence '\V'
<>:46: SyntaxWarning: invalid escape sequence '\V'
/tmp/ipykernel_19344/47474260.py:46: SyntaxWarning: invalid escape sequence '\V'
  print(f"\VERIFICA CONTEGGI:")


Trovati 25 file .parquet in '../Pre-Elaborazione Dati/Dataset'

FASE 1: Avvio scansione unificata (UN FILE ALLA VOLTA)...
  Processando blocco_000-segmentato.parquet...
  Processando blocco_001-segmentato.parquet...
  Processando blocco_002-segmentato.parquet...
  Processando blocco_003-segmentato.parquet...
  Processando blocco_004-segmentato.parquet...
  Processando blocco_005-segmentato.parquet...
  Processando blocco_006-segmentato.parquet...
  Processando blocco_007-segmentato.parquet...
  Processando blocco_008-segmentato.parquet...
  Processando blocco_009-segmentato.parquet...
  Processando blocco_010-segmentato.parquet...
  Processando blocco_011-segmentato.parquet...
  Processando blocco_012-segmentato.parquet...
  Processando blocco_013-segmentato.parquet...
  Processando blocco_014-segmentato.parquet...
  Processando blocco_015-segmentato.parquet...

------------------------------------------------------------
FASE 1: Scansione completata.
FASE 2: Verifica e Analisi.
\VERIF

### Normalizzazione

Passaggio fondamentale perchè le tue reti neurali (LSTM/LNN) faticano a imparare quando i dati di input hanno scale completamente diverse. Ad esempio, la colonna COG (rotta) va da 0 a 360, mentre la colonna SOG (velocità) potrebbe andare da 0 a 50. Questi intervalli così diversi "confondono" la rete durante l'addestramento.
Dobbiamo quindi standardizzarli, portando tutte le colonne a una scala simile.  
Il modo corretto per standardizzare è calcolare la media e la deviazione standard dell'intero set di addestramento (tutti i 4.3 milioni di traiettorie nei tuoi 16 file). Ovviamente, questi dati sono troppo grandi per essere caricati tutti insieme in memoria.

Quello che facciamo è:  
- Inizializziamo uno scaler vuoto e iteriamo sui 16 file di Train.
- `partial_fit` è la parte fondamentale perchè per ogni file che carica. Questa funzione dice allo scaler: "Prendi la media e la deviazione che hai calcolato finora e aggiornale includendo questo nuovo blocco di dati".
- `joblib.dump(scaler, SCALER_PATH)` salva questi parametri calcolati in un file esterno.


In [ ]:
COLONNE_DA_NORMALIZZARE = ['Latitude', 'Longitude', 'SOG', 'COG']
SCALER_PATH = 'scaler.joblib'

scaler = StandardScaler()

try:
    
    for i, file_path in enumerate(TRAIN_FILES):
        
        print(f"  Processando file {i+1}/{len(TRAIN_FILES)}: {os.path.basename(file_path)}...")
        
        df_chunk = pd.read_parquet(file_path, columns=COLONNE_DA_NORMALIZZARE)
        
        scaler.partial_fit(df_chunk)
        
        del df_chunk
        gc.collect()

    print("\nAdattamento completato su tutti i 16 file di training.")
    
    joblib.dump(scaler, SCALER_PATH) #Salva file

    print(f"\nScaler salvato come '{SCALER_PATH}'.")

except Exception as e:
    print(f"\n Errore durante la preparazione dello scaler: {e}")

  Processando file 1/16: blocco_000-segmentato.parquet...
  Processando file 2/16: blocco_001-segmentato.parquet...
  Processando file 3/16: blocco_002-segmentato.parquet...
  Processando file 4/16: blocco_003-segmentato.parquet...
  Processando file 5/16: blocco_004-segmentato.parquet...
  Processando file 6/16: blocco_005-segmentato.parquet...
  Processando file 7/16: blocco_006-segmentato.parquet...
  Processando file 8/16: blocco_007-segmentato.parquet...
  Processando file 9/16: blocco_008-segmentato.parquet...
  Processando file 10/16: blocco_009-segmentato.parquet...
  Processando file 11/16: blocco_010-segmentato.parquet...
  Processando file 12/16: blocco_011-segmentato.parquet...
  Processando file 13/16: blocco_012-segmentato.parquet...
  Processando file 14/16: blocco_013-segmentato.parquet...
  Processando file 15/16: blocco_014-segmentato.parquet...
  Processando file 16/16: blocco_015-segmentato.parquet...

Adattamento completato su tutti i 16 file di training.

✅ SUCCES